In [245]:
import sys
import os
from crewai import Process, Crew, Agent, Task
from crewai_tools import BaseTool, SerperDevTool
import requests
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

In [246]:
load_dotenv()
search_tool = SerperDevTool()

1. Knowledge Retrieval Agent

In [247]:
knowledge_agent = Agent(
    role="AWS Documentation Specialist",
    goal="Retrieve and synthesize comprehensive AWS documentation tailored specifically to the {use_case}",
    backstory=(
        "You are an AWS certified Solutions Architect with 10+ years of experience specializing in AWS documentation. "
        "You're tasked with gathering detailed information for a {cost_profile} solution with {scalability} scalability needs "
        "and {security_tier} security requirements that must achieve {performance} performance while maintaining {availability}. "
        "The solution must address the specific challenges of implementing a {use_case}. "
        "You understand that this {implementation_time} timeline project requires {ease_of_implementation} implementation with "
        "{required_expertise} AWS expertise level. The solution must comply with {compliance} requirements. "
        "Your job is to find and extract the most relevant AWS documentation, white papers, implementation guides, and "
        "reference architectures that specifically address these exact requirements for this use case."
    ),
    allow_delegation=False,
    verbose=True,
)

2. Industry Solutions Expert Agent 

In [248]:
industry_solutions_agent = Agent(
    role="Industry Solutions Expert",
    goal="Identify industry-specific AWS patterns and real-world case studies that align with implementing {use_case}",
    backstory=(
        "You are an AWS Technical Account Manager specializing in {use_case} implementations. "
        "You have extensive experience with projects that require {cost_profile} optimization, "
        "{scalability} scaling capabilities, and {security_tier} security controls. "
        "Your expertise is particularly valuable for implementations needing {availability} across regions "
        "while meeting {performance} requirements. You focus exclusively on how real companies have implemented "
        "similar solutions that had to comply with {compliance} regulations. You understand the practical challenges "
        "of achieving {ease_of_implementation} implementation with {integration_complexity} integration complexity "
        "given {required_expertise} expertise level, especially when working within a {implementation_time} timeframe. "
        "Your value comes from finding concrete examples that match these exact parameters rather than generic solutions."
    ),
    allow_delegation=True,
    verbose=True,
)

3. Implementation Details Agent 

In [249]:
implementation_details_agent = Agent(
    role="Implementation Architect",
    goal="Create detailed implementation specifications and code examples for a {use_case} with {cost_profile} cost profile",
    backstory=(
        "You are a hands-on AWS implementation specialist who excels at developing {security_tier} secure solutions "
        "that deliver {performance} performance with {availability} reliability. Your specialty is converting high-level "
        "architectures into executable code, especially for {use_case} implementations. "
        "You understand that the client needs {ease_of_implementation} implementation complexity with {integration_complexity} "
        "integration requirements, given their {required_expertise} AWS expertise level. "
        "Your implementations must scale to {scalability} levels and be deployable within a {implementation_time} timeline. "
        "Every solution you create includes detailed IAM policies, networking configurations, and service settings "
        "tailored to meet {compliance} requirements. You search for and adapt real-world code examples from GitHub "
        "and AWS samples that match these exact requirements, providing implementation steps that are appropriate for "
        "the client's expertise level while meeting all their technical requirements."
    ),
    allow_delegation=True,
    verbose=True,
)

4. Technical Constraints Specialist Agent

In [250]:
constraints_agent = Agent(
    role="Technical Constraints Specialist",
    goal="Identify implementation limitations and challenges for {use_case} with {security_tier} security and {compliance} compliance",
    backstory=(
        "You are an AWS technical specialist who focuses exclusively on practical limitations and edge cases that "
        "impact real-world {use_case} implementations. You have deep knowledge of how service quotas and technical "
        "constraints affect solutions that require {scalability} scalability and {performance} performance while "
        "maintaining {availability}. You understand the specific challenges of implementing {cost_profile} solutions "
        "that must meet {security_tier} security standards and {compliance} compliance requirements. "
        "Your expertise is particularly valuable for projects with {implementation_time} timelines and teams with "
        "{required_expertise} expertise levels who need {ease_of_implementation} implementation approaches. "
        "For each component in an architecture, you identify quotas, limits, integration challenges, and "
        "region-specific constraints that could affect implementation success, along with specific workarounds."
    ),
    allow_delegation=True,
    verbose=True,
)

5. QA and Validation Agent

In [251]:
qa_validation_agent = Agent(
    role="Solution Validation Expert",
    goal="Critically evaluate {use_case} architectures against {cost_profile}, {scalability}, {security_tier}, and {performance} requirements",
    backstory=(
        "You are a senior AWS consultant who specializes in pre-implementation validation for {use_case} solutions. "
        "Your responsibility is to identify gaps or suboptimal choices in proposed architectures, ensuring they truly "
        "deliver {performance} performance with {availability} while maintaining {cost_profile} cost efficiency. "
        "You have deep expertise in evaluating solutions against {compliance} requirements and {security_tier} security standards. "
        "You understand the practical implementation challenges teams with {required_expertise} expertise face "
        "when implementing solutions with {integration_complexity} integration complexity, especially on "
        "{implementation_time} timelines. You're not satisfied with generic 'best practices' - you challenge every "
        "architectural decision to ensure it's the optimal choice for these specific requirements."
    ),
    allow_delegation=True,
    verbose=True,
)

6. Final Architecture Synthesizer Agent

In [252]:
architecture_synthesizer = Agent(
    role="Lead Solutions Architect",
    goal="Create an implementation-ready AWS architecture document for {use_case} with detailed deployment instructions",
    backstory=(
        "You are a senior AWS Solutions Architect specializing in {use_case} implementations that require "
        "{performance} performance and {availability} availability. You excel at creating {cost_profile} solutions "
        "that can scale to {scalability} levels while maintaining {security_tier} security standards and meeting "
        "{compliance} compliance requirements. You understand the challenges of delivering solutions with "
        "{ease_of_implementation} implementation complexity and {integration_complexity} integration needs, "
        "especially for teams with {required_expertise} expertise working on {implementation_time} timelines. "
        "Your implementation documents include actual code examples, detailed configuration guides, step-by-step "
        "deployment instructions, and operational guidance that transforms architectural concepts into executable reality. "
        "Your solutions are consultant-grade, providing all the detail necessary for successful implementation rather "
        "than just high-level guidance."
    ),
    allow_delegation=False,
    verbose=True,
)

Task 1: Knowledge Retrieval and Synthesis

In [253]:
task_knowledge_retrieval = Task(
    description=(
        "Research and synthesize AWS documentation specifically for implementing a {use_case} that meets these requirements:\n"
        "- Cost profile: {cost_profile}\n"
        "- Scalability needs: {scalability}\n"
        "- Security requirements: {security_tier}\n"
        "- Performance needs: {performance}\n"
        "- Availability requirements: {availability}\n"
        "- Compliance requirements: {compliance}\n"
        "- Implementation timeline: {implementation_time}\n"
        "- Implementation ease: {ease_of_implementation}\n"
        "- Integration complexity: {integration_complexity}\n"
        "- Team expertise level: {required_expertise}\n\n"
        "For each service or architectural component you recommend for this specific use case and requirements:\n"
        "1. Document its specific purpose in this {use_case} architecture\n"
        "2. Detail how it meets the {cost_profile}, {scalability}, {security_tier}, and {performance} requirements\n"
        "3. Identify specific configuration options needed for this implementation\n"
        "4. Provide direct links to AWS documentation sections that address this exact use case and requirements\n"
        "Your output should be structured as implementation-focused knowledge, not a generic overview."
    ),
    expected_output=(
        "A comprehensive knowledge base specifically for {use_case} implementation that includes:\n"
        "- Core AWS services recommended for {use_case} with specific configuration details\n"
        "- Integration patterns between services optimized for {cost_profile} and {performance}\n"
        "- Security configurations that satisfy {security_tier} requirements and {compliance} compliance\n"
        "- Scaling mechanisms to achieve {scalability} and {availability} requirements\n"
        "- Implementation approaches suitable for {required_expertise} teams on {implementation_time} timelines\n"
        "- Direct references to specific sections in AWS documentation"
    ),
    agent=knowledge_agent,
    tools=[search_tool],
)

Task 2: Industry Solutions Analysis

In [254]:
task_industry_solutions = Task(
    description=(
        "Research and analyze real-world implementations of {use_case} solutions that meet these specific requirements:\n"
        "- Cost profile: {cost_profile}\n"
        "- Scalability needs: {scalability}\n"
        "- Security requirements: {security_tier}\n"
        "- Performance needs: {performance}\n"
        "- Availability requirements: {availability}\n"
        "- Compliance requirements: {compliance}\n"
        "- Implementation timeline: {implementation_time}\n\n"
        "For each relevant case study or reference architecture you find:\n"
        "1. Document the exact AWS services and configurations used\n"
        "2. Detail how the solution addressed requirements similar to our {cost_profile}, {scalability}, {security_tier} needs\n"
        "3. Extract specific implementation decisions that enabled success for {use_case} implementations\n"
        "4. Identify any optimization patterns that improved cost efficiency or performance\n"
        "5. Note any implementation challenges faced and how they were overcome\n"
        "Focus exclusively on finding concrete examples, not theoretical architectures."
    ),
    expected_output=(
        "A detailed analysis of real-world {use_case} implementations that includes:\n"
        "- Specific customer case studies of {use_case} implementations\n"
        "- Technical architecture details from these implementations\n"
        "- How these implementations achieved {cost_profile}, {scalability}, and {security_tier} requirements\n"
        "- Configuration choices made to meet {performance} and {availability} needs\n"
        "- Implementation approaches used by teams with {required_expertise} expertise levels\n"
        "- Specific lessons learned and optimization techniques from these implementations"
    ),
    agent=industry_solutions_agent,
    tools=[search_tool],
)

Task 3: Implementation Details Specification

In [255]:
task_implementation_details = Task(
    description=(
        "Develop detailed implementation specifications for a {use_case} solution that meets these requirements:\n"
        "- Cost profile: {cost_profile}\n"
        "- Scalability needs: {scalability}\n"
        "- Security tier: {security_tier}\n"
        "- Performance needs: {performance}\n"
        "- Availability requirements: {availability}\n"
        "- Compliance requirements: {compliance}\n"
        "- Implementation timeline: {implementation_time}\n"
        "- Team expertise level: {required_expertise}\n\n"
        "For each major component in the {use_case} architecture:\n"
        "1. Provide actual infrastructure-as-code templates (CloudFormation, Terraform, or CDK)\n"
        "2. Include specific AWS CLI commands or SDK code snippets for setup and configuration\n"
        "3. Detail IAM policies and security configurations that meet {security_tier} requirements\n"
        "4. Provide network configuration details optimized for {performance} and {availability}\n"
        "5. Include service-specific parameter configurations appropriate for {scalability} needs\n"
        "Your output must include actual code examples and step-by-step setup instructions."
    ),
    expected_output=(
        "A detailed implementation guide for {use_case} that includes:\n"
        "- Infrastructure-as-code templates for all major components\n"
        "- AWS CLI commands and SDK code snippets for configuration\n"
        "- Complete IAM policy documents meeting {security_tier} requirements\n"
        "- Security group and networking configurations for {performance} and {availability}\n"
        "- Service parameter settings optimized for {cost_profile} and {scalability}\n"
        "- Step-by-step deployment instructions appropriate for {required_expertise} teams"
    ),
    agent=implementation_details_agent,
    tools=[search_tool],
)

Task 4: Technical Constraints Analysis

In [256]:
task_constraints_analysis = Task(
    description=(
        "Identify all technical constraints and limitations for a {use_case} implementation with these requirements:\n"
        "- Cost profile: {cost_profile}\n"
        "- Scalability needs: {scalability}\n"
        "- Security tier: {security_tier}\n"
        "- Performance needs: {performance}\n"
        "- Availability requirements: {availability}\n"
        "- Compliance requirements: {compliance}\n"
        "- Implementation timeline: {implementation_time}\n\n"
        "For each component in the {use_case} architecture:\n"
        "1. Document applicable service quotas and limits that could affect implementation\n"
        "2. Identify potential performance bottlenecks for achieving {performance} requirements\n"
        "3. Detail integration challenges between services, especially for {availability} requirements\n"
        "4. Outline region-specific considerations for {availability} and {compliance} requirements\n"
        "5. Document specific technical constraints related to {compliance} implementation\n"
        "For each constraint identified, provide specific workarounds or solutions."
    ),
    expected_output=(
        "A comprehensive constraints analysis for {use_case} implementation that includes:\n"
        "- Service quotas and limits for each component that could affect {scalability}\n"
        "- Performance constraints and solutions to achieve {performance} requirements\n"
        "- Integration challenges and workarounds for {security_tier} and {availability} needs\n"
        "- Region-specific implementation details for {availability} requirements\n"
        "- Technical constraints related to {compliance} implementation\n"
        "- Specific solutions and workarounds for each constraint"
    ),
    agent=constraints_agent,
    tools=[search_tool],
)

Task 5: Solution Validation and Enhancement

In [257]:
task_validation = Task(
    description=(
        "Critically evaluate the proposed {use_case} architecture against these specific requirements:\n"
        "- Cost profile: {cost_profile}\n"
        "- Scalability needs: {scalability}\n"
        "- Security tier: {security_tier}\n"
        "- Performance needs: {performance}\n"
        "- Availability requirements: {availability}\n"
        "- Compliance requirements: {compliance}\n"
        "- Implementation timeline: {implementation_time}\n"
        "- Implementation ease: {ease_of_implementation}\n"
        "- Integration complexity: {integration_complexity}\n"
        "- Team expertise level: {required_expertise}\n\n"
        "For each component and design decision in the architecture:\n"
        "1. Verify alignment with the stated requirements\n"
        "2. Evaluate against the AWS Well-Architected Framework principles\n"
        "3. Identify any suboptimal choices for this specific {use_case}\n"
        "4. Suggest specific improvements that better meet the requirements\n"
        "5. Validate that implementation details are complete and technically sound\n"
        "Provide specific, actionable feedback focused on practical implementation concerns."
    ),
    expected_output=(
        "A detailed validation report that includes:\n"
        "- Analysis of how each component meets the specific {use_case} requirements\n"
        "- Well-Architected Framework evaluation focused on {cost_profile}, {security_tier}, and {performance}\n"
        "- Identification of suboptimal choices for {use_case} with {scalability} and {availability} needs\n"
        "- Specific architectural improvements with implementation detail\n"
        "- Implementation detail verification for {required_expertise} teams within {implementation_time} timelines\n"
        "- Actionable enhancements to improve the architecture"
    ),
    agent=qa_validation_agent,
    tools=[search_tool],
)

Task 6: Final Architecture Synthesis

In [258]:
task_architecture_synthesis = Task(
    description=(
        "Create a comprehensive, implementation-ready AWS architecture document for a {use_case} that meets these requirements:\n"
        "- Cost profile: {cost_profile}\n"
        "- Scalability needs: {scalability}\n"
        "- Security tier: {security_tier}\n"
        "- Performance needs: {performance}\n"
        "- Availability requirements: {availability}\n"
        "- Compliance requirements: {compliance}\n"
        "- Implementation timeline: {implementation_time}\n"
        "- Implementation ease: {ease_of_implementation}\n"
        "- Integration complexity: {integration_complexity}\n"
        "- Team expertise level: {required_expertise}\n\n"
        "The document must include:\n"
        "1. Executive summary relating the architecture directly to {use_case} requirements\n"
        "2. Detailed architecture diagram with all components and their interactions\n"
        "3. Component-by-component implementation specifications including service configurations, "
        "   integration details, security implementations, and scaling mechanisms\n"
        "4. Step-by-step implementation plan with actual code examples and configuration files\n"
        "5. Implementation timeline and resource requirements aligned with {implementation_time} needs\n"
        "6. Operational considerations and monitoring setup\n"
        "7. Cost estimation based on the {cost_profile} requirements\n"
        "The document should be structured as a consultant's implementation guide that provides all "
        "details needed for a {required_expertise} team to implement this solution."
    ),
    expected_output=(
        "A comprehensive AWS architecture implementation document for {use_case} that includes:\n"
        "- Executive summary targeted to the specific requirements\n"
        "- Detailed architecture diagram showing all components\n"
        "- Component-by-component specifications meeting {security_tier}, {performance}, and {scalability} needs\n"
        "- Implementation code and configuration files\n"
        "- Step-by-step deployment guide appropriate for {required_expertise} expertise\n"
        "- Implementation timeline aligned with {implementation_time} requirements\n"
        "- Operational guidance for maintaining {availability} requirements\n"
        "- Cost estimation based on {cost_profile} parameters\n"
        "The document will be implementation-ready with all details needed for deployment."
    ),
    agent=architecture_synthesizer
)

In [259]:
crew = Crew(
    agents=[
        knowledge_agent,
        industry_solutions_agent,
        implementation_details_agent,
        constraints_agent,
        qa_validation_agent,
        architecture_synthesizer
    ],
    tasks=[
        task_knowledge_retrieval,
        task_industry_solutions,
        task_implementation_details,
        task_constraints_analysis,
        task_validation,
        task_architecture_synthesis
    ],
    process=Process.sequential,  # Ensures logical step-by-step execution
    verbose=2,  # Set verbosity level for detailed logs during execution
)

2025-04-04 21:39:39,812 - 14928 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [ ]:
user_requirements = {
    'cost_profile': "Cost-Optimized",
    'scalability': "Medium",
    'ease_of_implementation': "Easy",
    'integration_complexity': "Moderate",
    'required_expertise': "Intermediate",
    'use_case': "AI-powered customer support chatbot",
    'security_tier': "Enterprise",
    'performance': "High Performance",
    'availability': "High Availability (multi-region)",
    'compliance': ["GDPR", "HIPAA"],
    'implementation_time': "Short (days)"
}

crew.kickoff(inputs=user_requirements)

 [DEBUG]: == Working Agent: AWS Documentation Specialist
 [INFO]: == Starting Task: Research and synthesize AWS documentation specifically for implementing a AI-powered customer support chatbot that meets these requirements:
- Cost profile: Cost-Optimized
- Scalability needs: Medium
- Security requirements: Enterprise
- Performance needs: High Performance
- Availability requirements: High Availability (multi-region)
- Compliance requirements: ['GDPR', 'HIPAA']
- Implementation timeline: Short (days)
- Implementation ease: Easy
- Integration complexity: Moderate
- Team expertise level: Intermediate

For each service or architectural component you recommend for this specific use case and requirements:
1. Document its specific purpose in this AI-powered customer support chatbot architecture
2. Detail how it meets the Cost-Optimized, Medium, Enterprise, and High Performance requirements
3. Identify specific configuration options needed for this implementation
4. Provide direct links to AWS

KeyboardInterrupt: 